In [1]:
import os

In [2]:
%pwd

'e:\\ai_projects\\End-to-End-CNN-Classifier-using-Mlflow-DVC\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\ai_projects\\End-to-End-CNN-Classifier-using-Mlflow-DVC'

Execute it on the terminal

```bash
export MLFLOW_TRACKING_URI=https://dagshub.com/ngamcode96/End-to-End-CNN-Classifier-using-Mlflow-DVC.mlflow
export MLFLOW_TRACKING_USERNAME=ngamcode96
export MLFLOW_TRACKING_PASSWORD=b7b5535539429903cc2867c516ca1623f436eb58

 ```

### Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_the_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

### Configuration Manager

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_path=CONFIG_FILE_PATH, params_path=PARAMS_FILE_PATH, secret_path = SECRET_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        self.secret = read_yaml(secret_path)
        

        create_directories([self.config.artifacts_root])
        
        #Evaluation config
    def get_evaluation_config(self) -> EvaluationConfig:
        model_training_config = self.config.model_training
        data_ingestion_config = self.config.data_ingestion
        mlflow_credentials = self.secret.mlflow_tracking_credentials
        
        os.environ['MLFLOW_TRACKING_URI'] = mlflow_credentials.MLFLOW_TRACKING_URI
        os.environ['MLFLOW_TRACKING_USERNAME'] = mlflow_credentials.MLFLOW_TRACKING_USERNAME
        os.environ['MLFLOW_TRACKING_PASSWORD'] = mlflow_credentials.MLFLOW_TRACKING_PASSWORD
                
        evaluation_config = EvaluationConfig(
            path_of_the_model=Path(model_training_config.trained_model_path),
            training_data=Path(data_ingestion_config.training_data_path),
            all_params=self.params,
            mlflow_uri=mlflow_credentials.MLFLOW_TRACKING_URI,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        
        return evaluation_config
        

### Component

In [7]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from cnnClassifier.utils.common import save_json

In [8]:
class ModelEvaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    
    def _valid_generator(self):
        
        data_generator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation="bilinear"
        )
        
        valid_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator_kwargs
        )
        
        self.valid_generator = valid_data_generator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_the_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(Path("scores.json"), data=scores)
    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")
        
        

### Pipeline

In [9]:
try:
    config = ConfigurationManager()
    evauation_config = config.get_evaluation_config()
    model_evaluation = ModelEvaluation(evauation_config)
    model_evaluation.evaluation()
    model_evaluation.save_score()
    model_evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2024-12-20 19:44:48,515: INFO: common: yaml file: config\config.yaml is loaded successfully]
[2024-12-20 19:44:48,518: INFO: common: yaml file: params.yaml is loaded successfully]
[2024-12-20 19:44:48,520: INFO: common: yaml file: secret.yaml is loaded successfully]
[2024-12-20 19:44:48,520: INFO: common: Created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 9s 1s/step - loss: 2.4808 - accuracy: 0.8235
[2024-12-20 19:44:58,517: INFO: common: json file saved at: scores.json]


2024/12/20 19:44:59 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-12-20 19:45:00,781: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
[2024-12-20 19:45:01,450: INFO: builder_impl: Assets written to: C:\Users\amado\AppData\Local\Temp\tmpthvc13lv\model\data\model\assets]


c:\Users\amado\AppData\Local\Programs\Python\Python310\lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/12/20 19:45:33 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 4
Created version '4' of model 'VGG16Model'.
